### 參考資料
* https://medium.com/@mengchiang000/redis-cache淺談-應用-7513a9f5f2dd
* http://tw511.com/18/146/4292.html
* https://www.runoob.com/w3cnote/python-redis-intro.html
* https://kknews.cc/zh-tw/code/ljx6xke.html

In [4]:
import redis # !pip install redis

In [5]:
# 讀取本地端起的 redis-server
# redis 取出的结果默认是字节，我们可以设定 decode_responses=True 改成字符串。
r = redis.Redis(host='127.0.0.1', port=6379, decode_responses=True)

In [6]:
r.set('mykey', 10) #新增
print(r.get('mykey')) #獲取

10


In [7]:
r.set('tmp', 1234, ex=3) # key是"tmp" value是1234。將鍵值存入緩存, ex 可以設定過期的時間(秒)

True

In [9]:
print(r.get('tmp'))  # tmp=3 這個值過3秒就會不見

None


* 計算雜湊值

In [10]:
#!/usr/bin/python3
import hashlib
m = hashlib.md5()
data = "G. T. Wang"

# 先將資料編碼，再更新 MD5 雜湊值
m.update(data.encode("utf-8"))

h = m.hexdigest()
print(h)

6871bb74e606dc7b4988b643e4870a8d


In [28]:
import random
account = str(random.randint(1,10))
m = hashlib.md5()
m.update(account.encode("utf-8"))
password = m.hexdigest()
print(account, password)

6 1679091c5a880faf6fb5e6087eb1b2dc


* pymysql 連進本地資料庫

In [34]:
import pymysql
import pandas as pd
conn = pymysql.connect(host='localhost', port=3306, user='root', passwd='', db= 'test', charset='utf8')
cursor = conn.cursor()
sql = f"""
SELECT *
FROM test.student
"""
df = pd.read_sql(sql, conn)
df

,course,mark,name
0,Maths,60,Roger
1,Maths,60,Jerry
2,Maths,70,Ariel
3,Maths,55,Tracey
4,Biology,60,Tracey
5,Biology,70,Jerry
6,Econ,90,Jerry
7,Econ,60,Ariel
8,123,456,789


In [33]:
sql = f"""
INSERT INTO test.student(course, mark, name) VALUES ("123", "456", "789")
"""
cursor.execute(sql)
conn.commit()

***

In [49]:
r.flushdb() # 實驗一開始先清除 redis cache
trun_cmd = f"""TRUNCATE TABLE test.tb_user"""
cursor.execute(trun_cmd) # 實驗一開始先清除 mysql database
conn.commit()

for i in range(1,100):
    account = str(random.randint(1,100))
    m = hashlib.md5()
    m.update(account.encode("utf-8"))
    password = m.hexdigest()
    
    # 先去 redis 找
    pw = r.get(account)
    if not pw:
        # redis 找不到就先去 mysql 找
        sql = f"""
        SELECT password FROM test.tb_user WHERE account={account}
        """
        df = pd.read_sql(sql, conn)
        if len(df)==0:
            # 如果 mysql 找不到，就寫一筆進去 database
            cmd = f"""
            INSERT INTO test.tb_user(account, password) VALUES ("{account}", "{password}")
            """
            cursor.execute(cmd)
            conn.commit()
            # 再寫一筆進去 redis
            r.set(account, password, ex=10000)
        else:
            # mysql 有找到，一樣要寫到 redis
            r.set(account, password, ex=10000)
    
    
    print(account, password)

1 c4ca4238a0b923820dcc509a6f75849b
60 072b030ba126b2f4b2374f342be9ed44
11 6512bd43d9caa6e02c990b0a82652dca
95 812b4ba287f5ee0bc9d43bbf5bbe87fb
1 c4ca4238a0b923820dcc509a6f75849b
50 c0c7c76d30bd3dcaefc96f40275bdc0a
96 26657d5ff9020d2abefe558796b99584
2 c81e728d9d4c2f636f067f89cc14862c
72 32bb90e8976aab5298d5da10fe66f21d
60 072b030ba126b2f4b2374f342be9ed44
40 d645920e395fedad7bbbed0eca3fe2e0
81 43ec517d68b6edd3015b3edc9a11367b
68 a3f390d88e4c41f2747bfa2f1b5f87db
13 c51ce410c124a10e0db5e4b97fc2af39
59 093f65e080a295f8076b1c5722a46aa2
48 642e92efb79421734881b53e1e1b18b6
74 ad61ab143223efbc24c7d2583be69251
8 c9f0f895fb98ab9159f51fd0297e236d
37 a5bfc9e07964f8dddeb95fc584cd965d
68 a3f390d88e4c41f2747bfa2f1b5f87db
24 1ff1de774005f8da13f42943881c655f
90 8613985ec49eb8f757ae6439e879bb2a
5 e4da3b7fbbce2345d7772b0674a318d5
88 2a38a4a9316c49e5a833517c45d31070
49 f457c545a9ded88f18ecee47145a72c0
72 32bb90e8976aab5298d5da10fe66f21d
91 54229abfcfa5649e7003b83dd4755294
29 6ea9ab1baa0efb9e19094440c317e2

In [50]:
trun_cmd = f"""TRUNCATE TABLE test.tb_user"""
cursor.execute(trun_cmd)
conn.commit()